In [1]:
# Remove %%capture to see installation output
# %%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install evaluate  # Install the evaluate library for metrics
!pip install rouge_score

import torch
import math
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate

# Initialize metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",  # or choose another model
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [2]:
# Prepare the model for PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,   # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

# Set the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Function to format prompts
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    contexts = examples["context"]
    responses = examples["response"]

    # Combine instruction and context
    convos = [{"role": "user", "content": f"Instruction: {instruction}\nContext: {context}"}
              for instruction, context in zip(instructions, contexts)]
    assistant_responses = [{"role": "assistant", "content": response} for response in responses]

    # Combine conversations and apply the chat template
    conversations = [{"conversations": [convo, assistant_response]}
                     for convo, assistant_response in zip(convos, assistant_responses)]

    texts = [tokenizer.apply_chat_template(convo["conversations"], tokenize=False, add_generation_prompt=False)
             for convo in conversations]

    # Return the new 'text' column along with the original columns
    return {
        "text": texts,
        "instruction": instructions,
        "context": contexts,
        "response": responses
    }

Unsloth 2024.11.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [3]:
# Load and split the dataset
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# Split dataset into training and validation sets
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

# Apply formatting to the training dataset and remove original columns
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset['train'].column_names
)

# Apply formatting to the evaluation dataset but keep original columns
eval_dataset = eval_dataset.map(
    formatting_prompts_func,
    batched=True
    # Do not specify remove_columns here
)

README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/13509 [00:00<?, ? examples/s]

Map:   0%|          | 0/1502 [00:00<?, ? examples/s]

In [4]:
# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Use this for WandB etc
        # evaluation_strategy="steps",
        # eval_steps=10,
    ),
)

# Adjust trainer to train on responses only
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

# Start training
trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/13509 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1502 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map:   0%|          | 0/13509 [00:00<?, ? examples/s]

Map:   0%|          | 0/1502 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 13,509 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,2.497500
2,1.695500
3,2.312700
4,2.100000
5,2.734300
6,2.497800
7,2.043700
8,2.133100
9,2.517900
10,1.924400


In [5]:
# Evaluate the model to get perplexity
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity}")

Perplexity: 6.94609155583418


In [6]:
# Prepare the model for inference
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# Function to generate predictions and compute metrics
def evaluate_model(model, tokenizer, eval_dataset, batch_size=2):
    from torch.utils.data import DataLoader
    predictions = []
    references = []

    model.eval()
    model.to("cuda")
    eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            # Access the necessary columns
            instructions = batch['instruction']
            contexts = batch['context']
            responses_text = batch['response']

            batch_messages = []
            for instruction, context in zip(instructions, contexts):
                message = [
                    {
                        "role": "user",
                        "content": f"Instruction: {instruction}\nContext: {context}"
                    }
                ]
                batch_messages.append(message)

            # Apply chat template and tokenize
            inputs = tokenizer.apply_chat_template(
                batch_messages,
                tokenize=True,
                add_generation_prompt=True,  # Must add for generation
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=max_seq_length,
            ).to("cuda")  # Move tensors to the appropriate device

            # Compute attention mask
            attention_mask = inputs.ne(tokenizer.pad_token_id).long().to("cuda")

            # Generate outputs from the model
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=128,
                temperature=0.9,
                top_p=0.7,
            )

            # Get the length of the inputs
            input_length = inputs.shape[1]

            # Decode the generated text
            generated_tokens = outputs[:, input_length:]
            decoded_outputs = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )

            # Append the generated text to predictions
            predictions.extend(decoded_outputs)
            # Append the reference outputs
            references.extend(responses_text)

    # Compute BLEU and ROUGE scores
    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)

    print(f"Perplexity: {perplexity}")
    print(f"BLEU score: {bleu_score['bleu']}")
    print(f"ROUGE-1: {rouge_score['rouge1']}")
    print(f"ROUGE-2: {rouge_score['rouge2']}")
    print(f"ROUGE-L: {rouge_score['rougeL']}")

# Evaluate the model
evaluate_model(model, tokenizer, eval_dataset)


100%|██████████| 751/751 [1:04:36<00:00,  5.16s/it]


Perplexity: 6.94609155583418
BLEU score: 0.06934426439836616
ROUGE-1: 0.20689474561361007
ROUGE-2: 0.0911336002685195
ROUGE-L: 0.17096665710793296
